In [ ]:
#note: must use a microsim kernel or a kernel that can load all necessary python modules

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import multiprocessing as mp

In [2]:
from microsim.bp_treatment_strategies import *

from microsim.outcome_model_repository import OutcomeModelRepository
from microsim.outcome_model_type import OutcomeModelType
from microsim.person import Person

from microsim.population import NHANESDirectSamplePopulation

from microsim.bp_treatment_strategies import SprintTreatment

from microsim.sim_settings import simSettings

from microsim.trials.trialset import TrialsetParallel, TrialsetSerial
from microsim.trials.trial_description import TrialDescription
from microsim.trials.logistic_regression_analysis import LogisticRegressionAnalysis
from microsim.trials.linear_regression_analysis import LinearRegressionAnalysis
from microsim.trials.outcome_assessor import OutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AttributeOutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AssessmentMethod
from microsim.trials.risk_filter import RiskFilter
from microsim.trials.trial_utils import get_analysis_name, randomizationSchema

In [3]:
#any microsim dir will work, just need to access the NHANES data
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [4]:
#at the beginning turn on pandarallel if you are planning to use TrialsetSerial
pandarallel.initialize(verbose=1) #microsim by default now does not initialize pandarallel
simSettings.pandarallelFlag = True #with this flag all new population instances will be set to use pandarallel

In [5]:
#with pandarallel on these two steps are faster
pop = NHANESDirectSamplePopulation(10000, 1999)
#alive, df = pop.advance_vectorized(30)

In [ ]:
for x in range(0,8):
    print(pop._people.iloc[x]._gcp, OutcomeAssessor([OutcomeAssessor.CI]).get_outcome(pop._people.iloc[x]))

In [ ]:
pop._people.iloc[7]._gcp, OutcomeAssessor([OutcomeAssessor.CI]).get_outcome(pop._people.iloc[7])

In [6]:
anyEvent2 = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI, OutcomeType.DEMENTIA, OutcomeAssessor.CI]))
cogEvent = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.CI,OutcomeType.DEMENTIA]))
vascularEventOrDeath = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI]))
anyEvent = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI, OutcomeType.DEMENTIA]))
death = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH]))
qalys = LinearRegressionAnalysis(AttributeOutcomeAssessor("_qalys", AssessmentMethod.SUM))
meanGCP = LinearRegressionAnalysis(AttributeOutcomeAssessor("_gcp", AssessmentMethod.MEAN))
lastGCP = LinearRegressionAnalysis(AttributeOutcomeAssessor("_gcp", AssessmentMethod.LAST))

In [7]:
#we will later define how many processes to launch, this is just for validation
#and so that we do not set the number of processes to a number greater than the number of available cores
#print("code has access to cores ",len(os.sched_getaffinity(0)))

In [8]:
#these are usually set from the input file, but to make things easier here 
#just set them on the notebook (which means no script will test for their meaningfulness)
inputSampleSizes = [1000,2000,5000] #for quick tests
#inputSampleSizes = [100, 200, 500, 1000, 5000, 10000, 15000, 20000]
inputDurations = [1,2] #for quick tests
#inputDurations = [3,5,10,15,20]
#inputDemThresholds = [2.4845839854531493e-08, 0.00018576417292080007, 0.0012917270937081809, 0.005870510161620921, 0.025739443157677927]
#inputCvThresholds = [1.167603052003119e-06, 0.0008193743487641601, 0.0026191105926681, 0.006091251406939853, 0.0132184645579298]
#inputDemThresholds = [2.4845839854531493e-08,0.00018576417292080007] #for quick tests
#inputCvThresholds = [1.167603052003119e-06,0.0008193743487641601] #for quick tests
inputDemThresholds = [2.4845839854531493e-08] #for quick tests
inputCvThresholds = [1.167603052003119e-06] #for quick tests
inputTrialsetSize = 3
inputProcesses = 3

In [9]:
#if you try running a TrialsetParallel instance with pandarallel on, you will get an exception
#but these could be included in the TrialsetParallel.run() method
simSettings.pandarallelFlag = False #any new population instance will have pandarallel off 
pop.use_pandarallel(False) #for mp to pickle this instance that used pandarallel, must change its attributes

In [10]:
%%time
#a drawback of including these thresholds outside of the trialset instances is that these cannot be parallelized
#so in every iteration, I have to wait until all trials of the trialset are done in order to proceed to the next
#iteration/trialset (=time lost)
#the more thresholds I have, the greater the time lost 
#but it is relative to have many trials the set has and how many cores I am using
results = None
for dem in inputDemThresholds:
    for cv in inputCvThresholds:
            myRiskFilter = RiskFilter({OutcomeModelType.DEMENTIA : dem , OutcomeModelType.CARDIOVASCULAR : cv})
            #this trial description will be the same for all trials in the trial set
            desc = TrialDescription(sampleSizes= inputSampleSizes,
                                                durations = inputDurations,
                                                inclusionFilter=myRiskFilter.exceedsThresholds,
                                                exclusionFilter=None,
                                                analyses=[anyEvent2,
                                                          cogEvent,
                                                        death,
                                                        anyEvent,
                                                        vascularEventOrDeath,
                                                        qalys,
                                                        meanGCP,
                                                        lastGCP],
                                                treatment=SprintTreatment(),
                                                randomizationSchema=randomizationSchema)
            #create the set
            trialset = TrialsetParallel(desc,
                                        pop,
                                        inputTrialsetSize,
                                        inputProcesses, #an extra argument for TrialsetParallel
                                        additionalLabels=myRiskFilter._outcomeRiskThresholds)
            #trialset = TrialsetSerial(desc,
            #                          pop,
            #                          inputTrialsetSize,
            #                          additionalLabels=myRiskFilter._outcomeRiskThresholds)
            if __name__ ==  '__main__': #launching processes with multiprocesses requires this
                if results is None:
                    results = trialset.run()
                else:
                    results = pd.concat([results,trialset.run()]).reset_index(drop=True)

starting trial 0 now
starting trial 1 now
starting trial 2 now


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarnin

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

...

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

ending trial 0 now
ending trial 1 now


/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

ending trial 2 now
CPU times: user 316 ms, sys: 70.4 ms, total: 387 ms
Wall time: 57.8 s


In [11]:
results

,reg,se,pvalue,absEffectSize,duration,sampleSize,outcome,analysis,OutcomeModelType.DEMENTIA,OutcomeModelType.CARDIOVASCULAR
0,-2.726118e-01,2.236793e-01,2.229346e-01,-0.022000,1,1000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
1,-1.239934e-01,1.502887e-01,4.093516e-01,-0.011000,1,2000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
2,-1.602031e-03,9.413235e-02,9.864215e-01,-0.000145,1,5000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
3,-1.287121e-01,2.271430e-01,5.709474e-01,-0.010000,2,1000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
4,-9.356359e-02,1.635931e-01,5.673704e-01,-0.007000,2,2000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
5,-1.002041e-01,1.027236e-01,3.293255e-01,-0.007616,2,5000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
6,-3.007542e-01,2.251722e-01,1.816595e-01,-0.024000,1,1000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001
7,-1.142585e-01,1.512891e-01,4.501091e-01,-0.010000,1,2000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001
8,1.096656e-02,9.505659e-02,9.081529e-01,0.000971,1,5000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001
9,-7.886376e-02,2.293859e-01,7.309938e-01,-0.006000,2,1000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001
